# Caffe2 Concepts


# Blobs and Workspace, Tensors


下面，我们通过搭建一个简单的网络，来简单了解caffe2的Blobs and Workspace, Tensors怎么用，以及之间的关系：
我们这个网络有三层：

1.一个全连接层 (FC)

2.一个s型激活层，带有一个Softmax

3.一个交叉熵loss层



# 1.定义数据，标签；并讲数据标签放入workspace中的“data”“label”中

In [36]:
from caffe2.python import workspace, model_helper
import numpy as np

# Create the input data
data = np.random.rand(16, 100).astype(np.float32)

# Create labels for the data as integers [0, 9].
label = (np.random.rand(16) * 10).astype(np.int32)

workspace.FeedBlob("data", data)
workspace.FeedBlob("label", label)


True

# 2.使用model_helper新建一个model

In [37]:
# Create model using a model helper
m = model_helper.ModelHelper(name="my_first_net")

ModelHelper将会创建两个相关的net：

1.初始化相关参数的网络(ref. init_net)

2.执行实际训练的网络(ref. exec_net)


# 3.新建FC层

## 3.1 新建operators之前，先定义w，b

In [38]:
weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10, 100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10, ])

## 3.2 给m这个net新建一个FC的operators

In [39]:
fc_1 = m.net.FC(["data", "fc_w", "fc_b"], "fc1")

# 4.新建激活层和softmax层

In [40]:
pred = m.net.Sigmoid(fc_1, "pred")
softmax, loss = m.net.SoftmaxWithLoss([pred, "label"], ["softmax", "loss"])

注意：

1.我们这里batch_size = 16，也就是一次16个samples同时训练；

2.我们这里只是创建模型的定义，下一步开始训练；

3.model会存储在一个protobuf structure中，我们可以通过以下命令查看model：

In [41]:
print(m.net.Proto())

name: "my_first_net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label"



**查看初始化的参数:**

In [42]:
print(m.param_init_net.Proto())

name: "my_first_net_init"
op {
  output: "fc_w"
  name: ""
  type: "XavierFill"
  arg {
    name: "shape"
    ints: 10
    ints: 100
  }
}
op {
  output: "fc_b"
  name: ""
  type: "ConstantFill"
  arg {
    name: "shape"
    ints: 10
  }
}



# 5.执行

## 5.1.运行一次参数初始化：

In [43]:
workspace.RunNetOnce(m.param_init_net)

True

## 5.2.创建训练网络

In [44]:
workspace.CreateNet(m.net)

True

## 5.3.训练网络

In [45]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

执行后，您可以检查存储在输出blob（包含张量，即numpy数组）中的结果：

In [46]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.0876148  0.07548364 0.09623323 0.12869014 0.08200206 0.10208801
  0.12435256 0.11641801 0.09138598 0.09573159]
 [0.08656767 0.08965836 0.10828611 0.1141852  0.09495509 0.08777543
  0.11474496 0.10817766 0.10231902 0.09333058]
 [0.08370095 0.07577838 0.10283273 0.12051839 0.09019332 0.1001257
  0.12431574 0.10195971 0.10198188 0.09859312]
 [0.08037042 0.08022888 0.1113458  0.12137544 0.09024613 0.09120934
  0.11656947 0.10384342 0.10305068 0.10176046]
 [0.0878759  0.07796903 0.10136125 0.12322115 0.0864681  0.09394205
  0.11819859 0.11712413 0.08619776 0.10764208]
 [0.08172415 0.07318036 0.0948612  0.13026348 0.09646249 0.10189744
  0.10387536 0.1055788  0.08963021 0.12252647]
 [0.07917343 0.08171824 0.11740194 0.11786009 0.09542355 0.10089807
  0.10691947 0.08718182 0.09749852 0.11592486]
 [0.09023243 0.08145688 0.09926067 0.11211696 0.09406143 0.09656373
  0.10264882 0.11141657 0.09514029 0.11710224]
 [0.08539043 0.08547576 0.10477988 0.11539709 0.08580235 0.09573507
  0.11900022 

# 6.反向传播

这个网络只包含前向传播，因此它不会学习任何东西。通过在正向传递中为每个运算符添加gradient operators来创建向后传递。

In [50]:
from caffe2.python import workspace, model_helper
import numpy as np

# Create the input data
data = np.random.rand(16, 100).astype(np.float32)

# Create labels for the data as integers [0, 9].
label = (np.random.rand(16) * 10).astype(np.int32)

workspace.FeedBlob("data", data)
workspace.FeedBlob("label", label)

# Create model using a model helper
m = model_helper.ModelHelper(name="my_first_net")


weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10, 100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10, ])


fc_1 = m.net.FC(["data", "fc_w", "fc_b"], "fc1")
pred = m.net.Sigmoid(fc_1, "pred")
softmax, loss = m.net.SoftmaxWithLoss([pred, "label"], ["softmax", "loss"])

# 反向传播 add ================
m.AddGradientOperators([loss])
# ============================

workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net)

# 可以查看加了反向传播之后，net中多出来三个operators的反向传播层
print(m.net.Proto())
#=========================================================

# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))


name: "my_first_net_4"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "lab